In [1]:
# import required packages
import cv2
from PIL import Image
import csv
import numpy as np
from csv import DictWriter
import piexif
import datetime
import time
import math
from gpx_converter import Converter
import pandas as pd
from convertbng.util import convert_bng, convert_lonlat, convert_etrs89_to_osgb36               
import pyproj
from fractions import Fraction

In [7]:

# Enter path of video file 
video_file_path = '/Users/matthewtoberman/Dropbox/TRITONIA/ROV (1)/ROV_ALD_ARM01_25OCT2022/4K/C0033.MP4'
# Enter destination folder for images
cropped_image_folder = '/Users/matthewtoberman/Dropbox/TRITONIA/ROV (1)/ROV_ALD_ARM01_25OCT2022/4K/C0033_IMAGES'
# enter path of data folder
USBL_data_path = '/Users/matthewtoberman/Dropbox/TRITONIA/ROV (1)/ROV_ALD_ARM01_25OCT2022/ROV_ARM01_DIVE06_25102022_VID_033.csv'

# Enter video file start time and end time and frame interval for frame grab 
start_time_mins=2
start_time_secs=0
end_time_mins=6
end_time_secs=0
frame_interval=25

def to_deg(value, loc):
    """convert decimal coordinates into degrees, munutes and seconds tuple
    Keyword arguments: value is float gps-value, loc is direction list ["S", "N"] or ["W", "E"]
    return: tuple like (25, 13, 48.343 ,'N')
    """
    if value < 0:
        loc_value = loc[0]
    elif value > 0:
        loc_value = loc[1]
    else:
        loc_value = ""
    abs_value = abs(value)
    deg =  int(abs_value)
    t1 = (abs_value-deg)*60
    min = int(t1)
    sec = round((t1 - min)* 60, 5)
    return (deg, min, sec, loc_value)


def change_to_rational(number):
    """convert a number to rantional
    Keyword arguments: number
    return: tuple like (1, 2), (numerator, denominator)
    """
    f = Fraction(str(number))
    return (f.numerator, f.denominator)


def set_gps_location(file_name, lat, lng, altitude):
    global exif_dict
    """Adds GPS position as EXIF metadata
    Keyword arguments:
    file_name -- image file
    lat -- latitude (as float)
    lng -- longitude (as float)
    altitude -- altitude (as float)
    """

    lat_deg = to_deg(lat, ["S", "N"])
    lng_deg = to_deg(lng, ["W", "E"])

    exiv_lat = (change_to_rational(lat_deg[0]), change_to_rational(lat_deg[1]), change_to_rational(lat_deg[2]))
    exiv_lng = (change_to_rational(lng_deg[0]), change_to_rational(lng_deg[1]), change_to_rational(lng_deg[2]))

    gps_ifd = {
        piexif.GPSIFD.GPSVersionID: (2, 0, 0, 0),
        piexif.GPSIFD.GPSAltitudeRef: 1,
        piexif.GPSIFD.GPSAltitude: change_to_rational(round(altitude,5)),
        piexif.GPSIFD.GPSLatitudeRef: lat_deg[3],
        piexif.GPSIFD.GPSLatitude: exiv_lat,
        piexif.GPSIFD.GPSLongitudeRef: lng_deg[3],
        piexif.GPSIFD.GPSLongitude: exiv_lng,
    }

    exif_dict = {"GPS": gps_ifd}
    exif_bytes = piexif.dump(exif_dict)
    piexif.insert(exif_bytes, file_name)

# import usbl data
USBL_df = pd.read_csv(USBL_data_path)


# format lat lon into decimal degrees
def format_hdg(latlon):
    return float(latlon.split(";")[0])+float((latlon.split(";")[1]))/60+float((latlon.split(";")[2][:-1]))/3600

USBL_df['Lat'] = USBL_df.apply(lambda row : format_hdg(row['Lat']), axis = 1)
USBL_df['Lon'] = USBL_df.apply(lambda row : format_hdg(row['Lon']), axis = 1)




# create date time
USBL_df['date_time'] = pd.to_datetime("10/25/2022"+ " " + USBL_df['Time'])
USBL_df = USBL_df.copy()
USBL_df.index = USBL_df['date_time']
USBL_df.sort_index(inplace=True, ascending=True)
USBL_df = USBL_df.resample('1S').mean()
USBL_df = USBL_df.interpolate()



## run extraction 
# Read the video from specified path this could clearly be changed to a file open box
cap = cv2.VideoCapture(video_file_path)  
# find length of entire video in frames
total_frame_number = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
# find frame of video
frame_rate=cap.get(cv2.CAP_PROP_FPS)
# print(frame_rate)

#  find the time at the end of video so can use this if entire video is required
video_end_time_mins,video_end_time_secs  = divmod(math.floor(total_frame_number * frame_rate), 60)



# find start and end frames coorresponding to start and end times
start_frame=math.floor((start_time_mins*60*frame_rate)+(start_time_secs*frame_rate))
end_frame=math.floor((end_time_mins*60*frame_rate)+(end_time_secs*frame_rate))

#  video start time format yyyy,dd,mm,HH,MM,SS - check time zone of video and make sure it matches USBL logs
video_start_time = USBL_df.index[0]+datetime.timedelta(minutes=(start_time_mins))+datetime.timedelta(seconds=(start_time_secs))
image_time = video_start_time 
# Loop through specified frames 
for i in range(start_frame, end_frame, frame_interval):
    # choose specified frame
    cap.set(cv2.CAP_PROP_POS_FRAMES,i)
    # read specified frame
    ret, frame = cap.read()
    # create time stamp for frame file name
    fractional_mins, whole_mins = math.modf((i/frame_rate)/60)
    # create and save jpegs
    image_file_name = cropped_image_folder + '/Frame_'+str(i) + '_' + str(image_time.minute) + 'mins_' + str(str(image_time.second) )  + 'secs' +'.jpg'
    cv2.imwrite(image_file_name,frame)

    # # find closest time and thus lat lon  and depth from usbl DF
    idx= USBL_df.index.get_indexer([image_time], method='nearest')

    image_lon = USBL_df['Lon'][idx][0]
    image_lat = USBL_df['Lat'][idx][0]
    image_depth = USBL_df['Depth'][idx][0]*(-1)

    print(image_lat)
    print(image_lon)
    print(image_depth)

    set_gps_location(image_file_name,image_lat,image_lon,image_depth)
    
    # take EXIF informatoin from ramdom GoPro image to match that expected by UWIS CamPositioner
    # exif_dict = {}
    exif_dict['Exif'] = {}
    # change date time entries of Exif data to be that of the extracted frames
    exif_dict['Exif'][36867] = (image_time.strftime("%Y:%m:%d %H:%M:%S"))
    exif_dict['Exif'][36868] = (image_time.strftime("%Y:%m:%d %H:%M:%S"))
    
    # stamp exif data to image
    exif_bytes = piexif.dump(exif_dict)
    im = Image.open(image_file_name)
    im.save(image_file_name, exif=exif_bytes)
    # increment time stamp by time ellapsed between frames
    image_time = image_time + datetime.timedelta(seconds=(frame_interval/frame_rate))

9.402201666666667
46.193630999999996
501.7
9.402200583333332
46.19363274999999
501.5
9.4021995
46.193634499999995
501.3
9.402204583333333
46.193632833333325
501.0
9.402209666666668
46.19363116666666
500.7
9.402203333333334
46.19363433333333
500.4
9.402197000000001
46.193637499999994
500.1
9.402206583333335
46.19363575
499.85
9.402216166666667
46.193633999999996
499.6
9.4022045
46.19364158333333
499.5
9.402192833333334
46.19364916666667
499.4
9.402199833333334
46.19364491666666
499.4
9.402206833333334
46.19364066666666
499.4
9.402198
46.19364616666666
499.4
9.402189166666666
46.19365166666666
499.4
9.402194083333333
46.19364508333333
499.45
9.402199
46.1936385
499.5
9.402197000000001
46.193642249999996
499.7
9.402195
46.193645999999994
499.9
9.402192833333334
46.193647416666664
500.1
9.402190666666668
46.19364883333333
500.3
9.402193416666666
46.19364899999999
500.35
9.402196166666666
46.19364916666667
500.4
9.402194
46.19365775
500.4
9.402191833333333
46.19366633333333
500.4
9.40218975

In [ ]:
## Make gpx file
# Read file into data frame
OE_ROV_df = pd.read_csv('/Users/matthewtoberman/Dropbox/TRITONIA/OCEAN_ECOLOGY/04_RAW Data/N1_NNSSR_0013.csv')
gpx_file_name = '/Users/matthewtoberman/Dropbox/TRITONIA/OCEAN_ECOLOGY/04_RAW Data/N1_NNSSR_0013.gpx'

# set required input projection
inProj =  pyproj.Proj("EPSG:25831")
# set required output projection
outProj  =  pyproj.Proj("EPSG:4326") # WGS84 in degrees and not EPSG:3857 in meters)
# make transformation
OE_ROV_df['latitude'],OE_ROV_df['longitude'] =pyproj.transform(inProj,outProj,OE_ROV_df['Focal point Easting'], OE_ROV_df['Focal point Northing'])
# make datestamp
OE_ROV_df['date_time'] =  OE_ROV_df['Date'] + " " + OE_ROV_df['Time']
# make gpx file
Converter.dataframe_to_gpx(input_df=OE_ROV_df,
                           lats_colname='latitude',
                           longs_colname='longitude',
                           times_colname='date_time',
                           alts_colname='ROV Depth Value',
                           output_file=gpx_file_name)


In [ ]:
import os
image_folder = cropped_image_folder
image_col=[]
image_row =[]
image_file_name=[]
image_count =0
file_list =  os.scandir(image_folder)  
for images in file_list:
    image_file = images.name
    if image_file[-3:] =="jpg":
        exif_dict = piexif.load(image_file)
        print(exif_dict['GPS'] )


        # image_col.append(int(image_file.split('-')[1]))
        # image_row.append(int(image_file.split('-')[2].split('.')[0]))

        # image_file_name.append(image_folder+ '/' + image_file)

        # image_row_col_df = pd.DataFrame({'name' : image_file_name,'col': image_col, 'row': image_row})